#### [Kaggle Titanic Competition](https://www.kaggle.com/competitions/titanic)
Score: 0.78229

In [1]:
import pandas as pd

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [3]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
# check data types and nulls
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [5]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [6]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [7]:
# drop less relevant data
X = train.drop(['PassengerId', 'Survived', 'Name', 'Ticket', 'Cabin'], axis=1)
Xt = test.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)
X.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,22.0,1,0,7.2500,S
1,1,female,38.0,1,0,71.2833,C
2,3,female,26.0,0,0,7.9250,S
3,1,female,35.0,1,0,53.1000,S
4,3,male,35.0,0,0,8.0500,S


In [8]:
# target data
y = train['Survived']

In [9]:
X.Age = X.Age.fillna(X.Age.median()) # fill empty lines with the median age value
X.Embarked = X.Embarked.fillna(X.Embarked.mode()) # fill with the most frequent values

Xt.Age = Xt.Age.fillna(X.Age.median()) # fill empty lines with the median age value
Xt.Fare = Xt.Fare.fillna(Xt.Fare.median()) # fill with the median fare value

In [10]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

for col in ['Sex', 'Embarked']:
    X[col] = le.fit_transform(X[col]) # transform categorical to numerical
    Xt[col] = le.transform(Xt[col])
X.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,1,22.0,1,0,7.2500,2
1,1,0,38.0,1,0,71.2833,0
2,3,0,26.0,0,0,7.9250,2
3,1,0,35.0,1,0,53.1000,2
4,3,1,35.0,0,0,8.0500,2


In [11]:
Xt.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,1,34.5,0,0,7.8292,1
1,3,0,47.0,1,0,7.0000,2
2,2,1,62.0,0,0,9.6875,1
3,3,1,27.0,0,0,8.6625,2
4,3,0,22.0,1,1,12.2875,2


In [12]:
# split data for cross validation
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

In [13]:
# XGBoost
from xgboost import XGBClassifier
xgb = XGBClassifier(random_state=42)

from sklearn.model_selection import cross_val_score
cross_val_score(xgb, X_train, y_train, cv=5).mean()

0.7929548387096774

In [14]:
# hyperparameter optimization
from sklearn.model_selection import GridSearchCV 
xgb = XGBClassifier(random_state=42)
params = {'n_estimators': [5, 10, 20, 30, 50, 100], 
          'max_depth': [2, 3, 5, 7, 9], 
          'learning_rate': [0.1, 0.2, 0.3, 0.4, 0.5]}
best_xgb2 = GridSearchCV(xgb, param_grid = params, cv = 5, verbose = True, n_jobs = -1)
best_xgb2 = best_xgb2.fit(X_train, y_train)
print(best_xgb2.best_score_)
print(best_xgb2.best_params_)

Fitting 5 folds for each of 150 candidates, totalling 750 fits
0.8314451612903225
{'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 50}


In [15]:
# prediction on test data
y_pred = best_xgb2.predict(Xt)

xgb_sub = {'PassengerId': test.PassengerId, 'Survived': y_pred}
sub_xgb = pd.DataFrame(data=xgb_sub)
sub_xgb.to_csv('xgb_sub.csv', index=False)

In [16]:
sub_xgb.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
